In [1]:
import kaggle
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import tree
from sklearn import ensemble
from sklearn.inspection import plot_partial_dependence
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
import pkbar

In [2]:
def name_to_status(df):
    title_dict = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"}
    
    titles = []
    for name in df['Name']:
        title = name.split(',')[-1].split('.')[0]
        titles.append(title[1:])
    df['Name'] = titles
    
    
    # a map of more aggregated title
    # we map each title
    df['Name'] = df.Name.map(title_dict)
    df.rename(columns = {'Name':'Status'}, inplace = True)
    return df

In [3]:
def add_ticket_type(df):
    
#     ticket_type_dict = {
#         'SCA4': 'A',
#         'SCA3': 'A',
#         'A4': 'A',
#         'A': 'A',
#         'AQ3': 'A',
#         'AQ4': 'A',
#         'SP': 'A',
#         'SOP': 'A',
#         'FA': 'A',
#         'SCOW': 'A',
#         'AS': 'A',
#         'SOPP': 'A',
#         'FC': 'A',
#         'SOTONO2': 'A', 
#         'CASOTON': 'A',
#         'A5': 'A',
#         'WC': 'A',
#         'SOTONOQ': 'A',
#         'STONOQ': 'A',
#         'PC': 'B',
#         'STONO2': 'B',
#         'PP': 'B',
#         'SCPARIS': 'B',
#         'CA': 'B',
#         'SOC': 'B',
#         'C': 'B',
#         'FCC': 'B',
#         'SWPP': 'B',
#         'SC': 'B',
#         'STONO': 'B',
#         'SCAH': 'B',
#         'WEP': 'B',
#         'PPP': 'B',
#         'LP': 'B',
#         'BASIC': 'C',
#     }

    ticket_type_dict = {
        'SCA4': 'A',
        'SCA3': 'A',
        'A4': 'A',
        'A': 'A',
        'AQ3': 'A',
        'AQ4': 'A',
        'SP': 'A',
        'SOP': 'A',
        'FA': 'A',
        'SCOW': 'A',
        'AS': 'A',
        'SOPP': 'A',
        'FC': 'A',
        'SOTONO2': 'A', 
        'CASOTON': 'A',
        'A5': 'A',
        'WC': 'A',
        'SOTONOQ': 'A',
        'STONOQ': 'A',
        'PC': 'A',
        'STONO2': 'A',
        'PP': 'A',
        'SCPARIS': 'A',
        'CA': 'A',
        'SOC': 'A',
        'C': 'A',
        'FCC': 'A',
        'SWPP': 'A',
        'SC': 'A',
        'STONO': 'A',
        'SCAH': 'A',
        'WEP': 'A',
        'PPP': 'A',
        'LP': 'A',
        'BASIC': 'C',
    }
        
    ticket_types = []
    for ticket in df['Ticket']:
        if pd.notna(ticket): 
            ticket_split = ticket.split(' ')
            if len(ticket_split) > 1:
                ticket_type = ticket_split[0]
                ticket_type = ticket_type.replace('.','')
                ticket_type = ticket_type.replace('/','')
                ticket_type = ticket_type.upper()
                ticket_types.append(ticket_type)
            else: ticket_types.append('BASIC')
        else: ticket_types.append(ticket)
    df['TicketType'] = ticket_types
#     df['TicketType'] = df.TicketType.map(ticket_type_dict)
    return df

In [4]:
# def add_ticket_type(df):
#     ticket_types = []
#     for ticket in df['Ticket']:
#         if pd.notna(ticket): 
#             ticket_split = ticket.split(' ')
#             if len(ticket_split) > 1:
#                 ticket_types.append(ticket_split[0])
#             else: ticket_types.append('basic')
#         else: ticket_types.append(ticket)
#     df['TicketType'] = ticket_types
# #     df.rename(columns = {'Ticket':'TicketType'}, inplace = True)
#     return df

In [5]:
def cabin_to_deck(df):
    decks = []
    for cabin in df['Cabin']:
        if pd.notna(cabin): decks.append(cabin[0])
        else: decks.append('Basic')
    df['Cabin'] = decks
    df.rename(columns = {'Cabin':'Deck'}, inplace = True)
    return df

In [355]:
def add_ticket_number(df):
    ticket_numbers = []
    for ticket in df['Ticket']:
        if ticket == 'LINE': ticket_numbers.append(0)
        elif pd.notna(ticket):
            ticket_split = ticket.split(' ')
            ticket_numbers.append(int(ticket_split[-1]))
        else: ticket_numbers.append(ticket)
    ticket_numbers_arr = np.array(ticket_numbers) #Experiment
    uni, count = np.unique(ticket_numbers_arr,return_counts=True)
    aux = uni[count < 3].tolist()
    for i, value in enumerate(ticket_numbers_arr):
        if value in aux:
            ticket_numbers_arr[i] = 0
    df['TicketNumber'] = ticket_numbers_arr #Experiment
    return df

In [356]:
def add_group_size(df):
    group_size = []
    for i, row in df.iterrows():
        group_size.append(len((df['TicketNumber'] == row['TicketNumber']).values.nonzero()[0]))
    df['GroupSize'] = group_size
    return df

In [357]:
def add_age_estimated(df):
    age_estimated = []
    for i in range(len(df)):
        if df['Age'].values[i] != df['Age'].values[i]:
            age_estimated.append(0)
        elif df['Age'].values[i] != int(df['Age'].values[i]) and df['Age'].values[i] > 1:
            age_estimated.append(1)
        else: 
            age_estimated.append(0)
    df['AgeEstimated'] = age_estimated
    return df

In [358]:
def add_fam_size(df):
    fam_size  = []
    for i in range(len(df)):
        fam_size.append(df['SibSp'][i]+df['Parch'][i])
    df['FamSize'] = fam_size
    return df

In [359]:
def add_number_of_friends(df):
    number_of_friends = []
    for i, row in df.iterrows():
        number_of_friends.append(row['GroupSize'] - row['FamSize'])
    df['NumberOfFriends'] = number_of_friends
    return df

In [360]:
def add_fare_per_person(df):
    fare_per_person = []
    for i in range(len(df)):
        fare_per_person.append(df['Fare'][i]/(df['FamSize'][i]+1))
    df['FarePerPerson'] = fare_per_person
    return df

In [361]:
def fill_ages(df, train_index):
    ages = []
    grouped_train = df.iloc[:train_index].groupby(['Sex','Pclass','Status'])
    grouped_median_train = grouped_train.median()
    grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Status', 'Age']]
    for i, row in df.iterrows():
        if row['Age'] != row['Age']: 
            condition = (
                (grouped_median_train['Sex'] == row['Sex']) & 
                (grouped_median_train['Status'] == row['Status']) & 
                (grouped_median_train['Pclass'] == row['Pclass'])
            ) 
            ages.append(grouped_median_train[condition]['Age'].values[0])
        else: ages.append(row['Age'])
    df['Age'] = ages
    return df

In [362]:
def add_age_times_class(df):
    age_times_class = []
    for i in range(len(df)):
        age_times_class.append(df['Pclass'][i]*df['Age'][i])
    df['AgeClass'] = age_times_class
    return df

In [363]:
def add_life_stage(df):
    minor = np.zeros(len(df), 'int')
    adult = np.zeros(len(df), 'int')
    elderly = np.zeros(len(df), 'int')

    for i, age in enumerate(df['Age'].values):
        if age < 18:
            minor[i] = 1
        elif age < 60:
            adult[i] = 1
        else:
            elderly[i] = 1

    df['Minor'] = minor
    df['Adult'] = adult
    df['Elderly'] = elderly
    return df

In [364]:
def add_fam_size_cat(df):
    singleton = np.zeros(len(df), 'int')
    small_fam = np.zeros(len(df), 'int')
    large_fam = np.zeros(len(df), 'int')

    for i, fam_size in enumerate(df['FamSize'].values):
        if fam_size == 1:
            singleton[i] = 1
        elif fam_size <= 4:
            small_fam[i] = 1
        else:
            large_fam[i] = 1

    df['Singleton'] = singleton
    df['SmallFam'] = small_fam
    df['LargeFam'] = large_fam
    return df

In [365]:
def categorify(df, columns):
    rectifier = 1
    for column in columns:
        category_dict = {}
        unique_categories = df[column].unique()
        unknown_index = (unique_categories != unique_categories).nonzero()[0]
        unique_categories = unique_categories.tolist()

        if len(unknown_index): 
            category_indexes = list(range(len(unique_categories)))
            unknown = unique_categories[unknown_index[0]]
            category_indexes[0] = unknown
            unique_categories.pop(unknown_index[0])
            unique_categories.insert(0,unknown)

        else:
            category_indexes = list(range(1,len(unique_categories)+1))

        for i in range(len(unique_categories)):
            category_dict[unique_categories[i]] = category_indexes[i]

        new_column = []
        for i in range(len(df)):
            new_value = category_dict[df[column][i]]
            if new_value == new_value: new_value = int(new_value)
            new_column.append(new_value)

        df[column] = new_column
#         df[column] = new_column.astype('category')
    return df

In [366]:
def fill_missing(df, ignore_columns):
    for column in df.columns:
        if column in ignore_columns: continue
        elif df[column].isna().any():
            median = df[column].describe()['50%']
            nan_indexes = df[column].isna().values
            new_column = np.array(df[column])
            new_column[nan_indexes] = median
#             new_column = new_column.astype('int')
            df[column] = new_column
            df[f'{column}Missing'] = nan_indexes 
    return df

In [367]:
def normalize(df, columns):
    for column in columns:
        df[column] = complete_df[column].values / complete_df[column].values.max()
    return df

In [368]:
def integerize(df, columns):
    for column in columns:
        df = df.astype({column: 'int'})
    return df

In [369]:
def metric(predictions, targets):
    return 100*(((predictions > 0) == targets).count_nonzero() / len(predictions))

In [370]:
def loss_func(predictions, targets):
    predictions = torch.sigmoid(predictions)
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [371]:
class DatasetTabular(Dataset):
    def __init__(self, df, target_key = None):
        self.df = df
#         self.cat = cat
#         self.cont = cont
#         self.cardinalities = [len(df[column].unique()) for column in cat]
#         self.cardinalities = [complete_df[column].nunique() for column in cat]
#         self.target_cardinality = None
        self.target_key = target_key
#         if target is not None: self.target_cardinality = len(df[target].unique()) - 1
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
#         vectors_to_concat = []
#         for j, column in enumerate(self.cat):
#             one_hot_vector = torch.zeros(self.cardinalities[j], dtype=torch.int)
#             one_hot_vector[self.df[column][i] - 1] = 1 
#             vectors_to_concat.append(one_hot_vector)
            
#         for column in self.cont:
#             vectors_to_concat.append(torch.tensor([self.df[column][i]]))

        input_vector_list = []
        for column in self.df:
            if column == self.target_key: continue
            else: input_vector_list.append(self.df[column].iloc[i])
        input_vector = torch.tensor(input_vector_list)
        
        if self.target_key is not None:
            target = torch.tensor([self.df[self.target_key][i]])
            return input_vector.float(), target
        
#         if self.target_cardinality is not None:
#             out_vector = torch.zeros(self.target_cardinality)
#             out_vector[self.df[self.target][i] - 1] = 1
#             return input_vector, out_vector
        
        return input_vector.float()

In [372]:
class LinearNet(nn.Module):
    def __init__(self, sizes):
        super(LinearNet, self).__init__()
        self.n_layers = len(sizes)-1
        self.linear = nn.ModuleList([nn.Linear(sizes[i], sizes[i+1]) for i in range(self.n_layers)])
        self.batchnorms = nn.ModuleList([nn.BatchNorm1d(sizes[i+1]) for i in range(self.n_layers)])
    
    def linear_block(self, x, idx):
        x = self.linear[idx](x)
        x = self.batchnorms[idx](x)
        if idx != self.n_layers-1:
            x = torch.sigmoid(x) 
        return  x
    
    def forward(self, x):
        for i in range(self.n_layers):
            x = self.linear_block(x,i)
        return x

In [373]:
class TrainAndEvaluate():
    def __init__(self, model, optimizer, loss_func, metric, train_dataloader, valid_dataloader):
        self.model = model
        self.optimizer = optimizer
        self.loss_func = loss_func
        self.train_dl = train_dataloader
        self.valid_dl = valid_dataloader
        self.metric = metric
    
    def train_and_evaluate(self, n_epoch):
        torch.autograd.set_detect_anomaly(True)

        for epoch in range(n_epoch):
            kbar = pkbar.Kbar(target=(len(self.train_dl)+len(self.valid_dl)), epoch=epoch, num_epochs=n_epoch, width=16, stateful_metrics=['Validation Loss','Accuracy'])
            running_loss_t = 0.0
            running_loss_v = 0.0
            
            for i, b in enumerate(self.train_dl):
                input_vector = b[0]
                target = b[1]
                preds = self.model(input_vector)
                
                self.optimizer.zero_grad()
                loss = self.loss_func(preds, target)
                
                loss.backward() 
                self.optimizer.step()
                running_loss_t = loss.item()
                kbar.update(i, values=[("Train Loss", running_loss_t)])
            
            kbar.add(1, values=[("Validation Loss", 0), ("Accuracy", 0)])        
            with torch.no_grad():
                for j, b in enumerate(self.valid_dl):
                    input_vector = b[0]
                    target = b[1]

                    preds = self.model(input_vector)
                    loss = self.loss_func(preds, target)
                    running_loss_v = loss
                    accuracy = self.metric(preds, target)
                    kbar.update(i+j, values=[("Validation Loss", running_loss_v), ("Accuracy", accuracy)])
        print('Finished Training')
        return self.model

In [374]:
# def rf_feat_importance(m, df):
#     return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
#     ).sort_values('imp', ascending=False)

In [375]:
# def one_hot_encode(df, key):
#     for i in range(len(df[key].unique())):
#         df[f'{key}_{i+1}'] = np.zeros(len(df), 'int')

#     for i in range(len(df)):
#         value = df[key].values[i]
#         df[f'{key}_{value}'][i] = 1
#     df = df.drop([key], axis=1)
#     return df

In [376]:
def one_hot_encode(df, key):
    array_list = []
    for i in range(len(df[key].unique())):
#         df[f'{key}_{i+1}'] = np.zeros(len(df), 'int')
        array_list.append(np.zeros(len(df),'int'))
        
    for i in range(len(df)):
        value = df[key].values[i]
        array_list[value-1][i] = 1
    
    for i in range(len(df[key].unique())):
        df[f'{key}_{i+1}'] = array_list[i]
        
    df = df.drop([key], axis=1)
    return df

In [377]:
path = '/home/francisco/workspace/titanic_kaggle/titanic'

In [378]:
original_train_df = pd.read_csv(f'{path}/train.csv',low_memory=False)
original_test_df = pd.read_csv(f'{path}/test.csv',low_memory=False)

In [379]:
complete_df = pd.concat([original_train_df, original_test_df], ignore_index=True)

In [380]:
complete_df = name_to_status(complete_df)
# complete_df = add_ticket_type(complete_df)
complete_df = add_ticket_number(complete_df)
# complete_df = add_group_size(complete_df)
complete_df = complete_df.drop(['Ticket'], axis=1)
complete_df = cabin_to_deck(complete_df)
complete_df = add_age_estimated(complete_df)
complete_df['AgeMissing'] = np.zeros(len(complete_df),'int')
complete_df['AgeMissing'][(complete_df['Age'] != complete_df['Age'])] = 1
complete_df = fill_ages(complete_df,891)
# complete_df = add_life_stage(complete_df)
complete_df = add_fam_size(complete_df)
# complete_df = add_number_of_friends(complete_df)
# complete_df = add_fam_size_cat(complete_df)
complete_df['Fare'] = complete_df['Fare'].fillna(complete_df['Fare'].median())
complete_df = add_fare_per_person(complete_df)
complete_df = complete_df.drop(['Fare'], axis=1)
complete_df = add_age_times_class(complete_df)
complete_df['Embarked'] = complete_df['Embarked'].fillna('S')

/tmp/ipykernel_50831/424082697.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_df['AgeMissing'][(complete_df['Age'] != complete_df['Age'])] = 1


In [381]:
# complete_df['TicketNumber'].value_counts()[:30]

In [382]:
# complete_df['FamSize'][complete_df['TicketNumber'] == 14879]

In [383]:
# complete_df['Survived'][complete_df['TicketNumber'] == 17608]

In [384]:
# complete_df['TicketType'][complete_df['TicketNumber'] == 17608]

In [385]:
complete_df = categorify(complete_df,['Status', 'Deck', 'Sex', 'Embarked', 'TicketNumber'])
complete_df = normalize(complete_df,['Age', 'FarePerPerson', 'SibSp' , 'Parch', 'FamSize',  'SibSp' , 'Parch', 'FamSize'])
# complete_df = fill_missing(complete_df, ['Survived'])

In [386]:
# complete_df = complete_df.drop(['TicketNumber'],axis=1)

In [387]:
for key in ['Sex', 'Pclass', 'Status', 'Deck', 'Embarked','TicketNumber']:
    complete_df = one_hot_encode(complete_df, key)

In [388]:
train_df = complete_df.iloc[:712]
valid_df = complete_df.iloc[712:891].reset_index()
test_df = complete_df.iloc[891:].reset_index()

In [389]:
train_dset = DatasetTabular(train_df.iloc[:,1:], 'Survived')
valid_dset = DatasetTabular(valid_df.iloc[:,2:], 'Survived')
test_dset = DatasetTabular(test_df.iloc[:,3:])

In [390]:
train_dl = DataLoader(train_dset, batch_size=256, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_dset, batch_size=179)

In [397]:
# train_dset[0][0].shape

In [398]:
# net = LinearNet([66,128,128,128,128,64,64,32,32,16,16,8,8,2,2,1])
# net = LinearNet([66,66,32,32,32,16,16,16,8,8,8,4,4,4,2,2,1])
# net = LinearNet([66,128,128,128,128,128,128,64,64,64,64,64,64,32,32,32,32,32,32,16,16,16,16,16,16,1])
# net = LinearNet([66,66,66,66,66,66,66,66,66,66,1])
# net = LinearNet([66,66,66,66,32,32,32,32,16,16,16,16,8,8,8,8,4,4,4,4,2,2,2,2,1])
# net = LinearNet([66,128,128,128,128,128,1])
# net = LinearNet([69,69,69,69,69,69,69,69,69,69,69,69,69,32,32,32,32,32,32,1])
net = LinearNet([116,116,64,64,32,32,1])

In [399]:
# optimizer = optim.SGD(net.parameters(), lr=0.00001)
optimizer = optim.AdamW(net.parameters(), lr=0.0001, weight_decay=10e-4)
trainer = TrainAndEvaluate(net,optimizer,loss_func,metric,train_dl,valid_dl)

In [400]:
net = trainer.train_and_evaluate(2000)

Epoch: 1/2000
1/3 [====>...........] - ETA: 2s - Train Loss: 0.3345 - Validation Loss: 0.3371 - Accuracy: 85.4749Epoch: 72/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.3251 - Validation Loss: 0.3339 - Accuracy: 86.0335Epoch: 143/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.3232 - Validation Loss: 0.3396 - Accuracy: 84.9162Epoch: 214/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.3052 - Validation Loss: 0.3379 - Accuracy: 85.4749Epoch: 285/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.3046 - Validation Loss: 0.3406 - Accuracy: 82.1229Epoch: 356/2000


...

1/3 [====>...........] - ETA: 1s - Train Loss: 0.2530 - Validation Loss: 0.3078 - Accuracy: 81.5642Epoch: 1765/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.2502 - Validation Loss: 0.3056 - Accuracy: 82.6816Epoch: 1835/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.2377 - Validation Loss: 0.3096 - Accuracy: 82.6816Epoch: 1905/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.2518 - Validation Loss: 0.3082 - Accuracy: 81.0056Epoch: 1975/2000


1/3 [====>...........] - ETA: 2s - Train Loss: 0.2340 - Validation Loss: 0.3059 - Accuracy: 82.1229Finished Training


88.8268156424581

In [56]:
# original_train_df

In [521]:
rf_feat_importance(rf,valid_df.iloc[:,3:])
# rf_feat_importance(rf,train_df.drop(['Survived'], axis=1))

,cols,imp
13,Status_1,0.200807
0,Sex,0.188715
9,AgeClass,0.080383
14,Status_2,0.068798
8,FarePerPerson,0.062767
15,Status_3,0.059696
12,Pclass_3,0.050893
4,TicketNumber,0.046302
7,FamSize,0.040919
19,Deck_1,0.035329


In [34]:
# valid_df['Sex'].hist()

In [86]:
# fig,ax = plt.subplots(figsize=(12, 4))
# plot_partial_dependence(rf, valid_df.iloc[:,3:], ['Age'],
# grid_resolution=20, ax=ax);

***

In [165]:
test_dl = DataLoader(test_dset,batch_size=418)

preds_test = net(next(iter(test_dl)))

preds_test[preds_test < 0] = 0
preds_test[preds_test > 0] = 1

preds_nn_df = pd.DataFrame()
preds_nn_df['PassengerId'] = test_df['PassengerId'].values
preds_nn_df['Survived'] = preds_test.int()

preds_nn_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [166]:
preds_nn_df.to_csv(f'{path}/submission_nn.csv',index=False)

***

In [54]:
preds_train = rf.predict(train_df.values[:,2:])

In [55]:
gt = train_df['Survived'].values

In [56]:
np.count_nonzero(preds_train == gt)/len(gt)

0.9876543209876543